# Basic Spline Fitting

## Load libraries

In [ ]:
import gamache as gm
import numpy as np
import scanpy as sc

## Load data

In [ ]:
adata = sc.datasets.paul15()

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.raw = adata.copy()

## Preprocessing

In [ ]:
sc.pp.recipe_zheng17(adata)

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=["paul15_clusters"], frameon=False)

In [ ]:
# Subset to [1Ery, 2Ery, 3Ery, 4Ery, 5Ery, 6Ery, 7MEP, 8Mk]
adata = adata[
    adata.obs["paul15_clusters"].isin(
        ["1Ery", "2Ery", "3Ery", "4Ery", "5Ery", "6Ery", "7MEP", "8Mk"]
    ),
    :,
]

In [ ]:
sc.tl.pca(adata, svd_solver="arpack")
sc.pp.neighbors(adata, n_neighbors=4, n_pcs=20)
sc.tl.umap(adata)

In [ ]:
sc.pl.umap(adata, color=["paul15_clusters"], frameon=False)

In [ ]:
sc.tl.diffmap(adata)

In [ ]:
DC1 = [first[0] for first in adata.obsm["X_diffmap"]]
DC2 = [first[1] for first in adata.obsm["X_diffmap"]]
DC_tmp = [first[0] + first[1] for first in adata.obsm["X_diffmap"]]

adata.uns["iroot"] = np.argsort(DC_tmp)[-1]

sc.tl.dpt(adata)

In [ ]:
sc.pl.umap(adata, color=["paul15_clusters", "dpt_pseudotime"], frameon=False)

In [ ]:
adata.write("paul15_endo.h5ad")

## Fit the GAM model

In [ ]:
adata = sc.read("paul15_endo.h5ad")
adata = adata.raw.to_adata()

In [ ]:
# Fast penalized frequentist fit
model = gm.tl.fit_gam(adata, backend="irls")

In [ ]:
gm.pl.plot_gene_fit(model=model, gene="Hba-a2")

In [ ]:
model.association_test(gene="Hba-a2")